In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from time import perf_counter

plt.rcParams.update({'font.family': 'cmr10', 'font.size': 12})
plt.rcParams['axes.unicode_minus'] = False
plt.rcParams['axes.labelsize'] = 12
plt.rcParams['figure.dpi'] = 100
plt.rcParams['mathtext.fontset'] = 'cm'
plt.rcParams['mathtext.rm'] = 'serif'
plt.rcParams['xtick.direction'] = 'in'
plt.rcParams['ytick.direction'] = 'in'
plt.rcParams['xtick.top'] = True
plt.rcParams['ytick.right'] = True
plt.rcParams['axes.formatter.use_mathtext'] = True

%matplotlib inline

In [ ]:
# Define a few useful functions 

def preview_img(index, data):
    """
    index: int, index of the image to display
    
    data: array, dataset to diplay an image from
    """
    if index > data.shape[0]:
        print(f"The index you choose must be less than {data.shape[0]}")
    else:
        fig, ax = plt.subplots(1,1, figsize=(4,4))
        ax.matshow(data[index,...], cmap='gray')
        ax.set_axis_off()
        
def model_accuracy(actual, predicted):
    correct = 0
    for i in range(len(actual)):
        if actual[i] == predicted[i]:
            correct += 1
    return correct / float(len(actual)) * 100

In [ ]:
# If you have not already installed TensorFlow and Keras, un-comment and run the following lines

#! pip install tensorflow
# If you have a Mac and the above line didn't work, try the one below
#! pip install tensorflow-macos

#! pip install keras

# Exploring and preprocessing our data

In [ ]:
# start by loading in data

from tensorflow.keras.datasets import mnist

(training_data, training_labels), (testing_data, testing_labels) = mnist.load_data()

print(training_data.shape, testing_data.shape)
print(training_labels.shape, testing_labels.shape)

By default, the label associated with each image is the value of the digit shown in that image. Since we're only interested in whether or not the handwritten digit is even or odd, we want to adjust the labels so that every even digit is labeled by a zero, and every odd digit is labeled by a one.

In [ ]:
training_labels[training_labels%2==0] = 0
training_labels[training_labels%2==1] = 1
testing_labels[testing_labels%2==0] = 0
testing_labels[testing_labels%2==1] = 1

With the labels sorted, let's make sure that the data matches what we expect. The training (testing) data contains 60,000 (10,000) 28x28 greyscale images of handwritten digits. Pick a few numbers between 0 and 59,999 and plug them into the <tt>preview_img</tt> function below.

In [ ]:
preview_img(12345, training_data)

For the network we'll be building today, we'll want to reformat each image as a column vector, and then stack all of those column vectors side-by-side into a 'matrix' that looks like $$\begin{bmatrix} | & | & | & \ldots & | \\ x^{[0]}_{i} & x^{[1]}_{i} & x^{[2]}_{i} & \ldots & x^{[N]}_{i} \\ | & | & | & \ldots & | \end{bmatrix}$$ where $x^{[k]}_{i}$ is the intensity of the $i$th pixel in the $k$th image, and $N$ is the total number of images we have (60,000 for the training set, 10,000 for the testing set)

There are network that allow us to train directly on images (Convolutional Neural Networks), but we won't be discussing those today.

In [ ]:
training_data = training_data.reshape(:20000, training_data.shape[1]*training_data.shape[2]).T
testing_data = testing_data.reshape(-1, testing_data.shape[1]*testing_data.shape[2]).T
training_labels = training_labels.reshape(1,:20000)
testing_labels = testing_labels.reshape(1,-1)

# Defining the Model Architecture

We can think of our neural network in the following way:

![](Images/black_box_nn.png)

where the inner workings are quite literally (for now, at least) a black box. 

Inside this model, there are a number of $\textit{layers}$, corresponding to either our input data, the model's output, or some learned representation along the way. We say a layer is "hidden" if it isn't the model's input or output, since we typically don't access any of the information in those layers. If it's helpful, you can think of these layers as being column vectors. Our updated picture now looks like:

![](Images/nn_arch.png)

Each $\textit{layer}$ consists of a certain number of $\textit{nodes}$. For our example, the number of nodes in the input layer will be the total number of pixels in the input image. We'll only have a single node in the output layer, since we want our model to learn whether a given input corresponds to one of two classes. 

The network we'll build today is called a $\textbf{Dense}$ network since every node in the $k$th layer depends on every node in the $k-1$th layer. Diagramatically, this looks like:

![](Images/dense_layout.png)

where each circle represents a node in a given layer, and each arrow indicates how much the node in one layer depends on the value of the node in the previous layer.

Mathematically, the value of the $i$th node in the $k$th layer is $$z^{[k]}_{i} = \displaystyle\sum_{j=1}^{N_{k-1}} w_{ij}z^{[k-1]}_{j} + b_{i}$$ where $w_{ij}$ is a matrix of weights connecting the $i$th node in the $k$th layer to all of the nodes in the $k-1$th layer, and $b_{i}$ is bias parameter that can also be learned. We can then do this operation for every node $i$ in layer $k$ by stacking them in a column. 

$$
\begin{bmatrix} x_{0}^{[k]} \\ x_{1}^{[k]} \\ \vdots \\ x_{N_{k}}^{[k]}\end{bmatrix} = 
\begin{bmatrix} w_{00} & w_{01} & \ldots & w_{0 N_{k-1}} \\ w_{10} & w_{11}  & \ldots &  w_{1 N_{k-1}} \\ \vdots & \vdots & \ddots & \vdots \\ w_{N_{k} 0} & w_{N_{k} 1} & \ldots &  w_{N_{k} N_{k-1}} \end{bmatrix}
\begin{bmatrix} x_{0}^{[k-1]} \\ x_{1}^{[k-1]} \\ \vdots \\ x_{N_{k-1}}^{[k-1]}\end{bmatrix} + 
\begin{bmatrix} b_{0}^{[k]} \\ b_{1}^{[k]} \\ \vdots \\ b_{N_{k}}^{[k]}\end{bmatrix}
$$

To summarize, the architecture of the network is determined by the the number of layers we use, and the number of nodes in each layer. The number of nodes in the input and output layers will be determined by our data and the task we ultimately want to do, but we are free to set the number of hidden layers as well as the number of nodes in each of those hidden layers. For simplicity, we will create a model with just a single hidden layer.

In [ ]:
training_labels.shape

In [ ]:
def nodes_per_layer(input_data, true_labels):
    """
    Define the number of nodes in each layer. The number of nodes in the input and output layer are determined 
    by our input data and true_labels, respectively. But, we are free to set the number of nodes 
    in the hidden layer.
    """
    n_inputs = input_data.shape[0]
    n_hidden = 12
    n_outputs = true_labels.shape[0]
    
    return (n_inputs, n_hidden, n_outputs)

In [ ]:
def initialize_parameters(n_inputs, n_hidden, n_outputs):
    """
    Initialize all of the parameters used to train the model, given the number of nodes in each layer. 
    W1 will connect our input layer to the hidden layer, W2 will connect the input layer to the output layer. 
    To start, we will set both bias vectors to 0.
    """
    W1 = np.random.randn(n_hidden, n_inputs)*0.01
    b1 = np.zeros((n_hidden,1))
    W2 = np.random.randn(n_outputs, n_hidden)*0.01
    b2 = np.zeros((n_outputs,1))
    
    parameters = {"W1":W1, "b1":b1, "W2":W2, "b2":b2}
    
    return parameters

# Forward Propagation

With the architecture of our model defined, we can take the first step towards training it. Starting from our input data, we'll carry out the necessary operations to produce the model's prediction for each input's corresponding label. 

The power of machine learning comes from being able to learn non-linearities present in our data. Our current formulation of connecting layers , however, only allows us to learn linear combinations of our data. To correct this, we then need to pass each layer (with the exception of the input layer) through a nonlinear function, called an $\textit{activation}$ function. 

For the hidden layer, we'll be using the <tt>tanh</tt> function, though there are other choices we could have made. For the output layer though, we want to make sure that we choose a function who's domain is $[0,1]$ since the labels we want our model to predict are either 0 or 1. We will use the <tt>sigmoid</tt> activation function for the output layer. Note that if we were interested in a different task, we would want to choose a different activation function for the output layer. 

In [ ]:
def sigmoid(x):
    
    s = 1/(1+np.exp(-x))
    
    return s

# plot the sigmoid function
x = np.linspace(-5,5,101)
plt.plot(x, sigmoid(x));

In [ ]:
def forward_prop(input_data, parameters):
    """
    We know how to link each layer to the next given the number of nodes in the input layer, 
    hidden layer, and output layer. We want to use the 'tanh' activation function for the hidden layer, and the
    'sigmoid' activation function for the output layer.
    """
    W1 = parameters["W1"]
    b1 = parameters["b1"]
    W2 = parameters["W2"]
    b2 = parameters["b2"]
    
    Z1 = np.dot(W1, input_data) + b1
    A1 = np.tanh(Z1)
    Z2 = np.dot(W2, A1) + b2
    A2 = sigmoid(Z2)
    
    cache = {"Z1":Z1, "A1":A1, "Z2":Z2, "A2":A2}
    
    return A2, cache

For each event, we compute the 
$\textit{loss}$, which tells us how far off the network's guess was from the true label. For binary classification, the loss function we often us is the $\textbf{binary crossentropy}$ $$\mathcal{L}(\hat{y}, y) = -y\ln(\hat{y}) - (1-y)\ln(1-\hat{y})$$ where $y$ is the true label and $\hat{y}$ is the network's output (what we have been calling $A^{[2]}$). By eye, we can see that if the network's output matches the true label, the loss function will evaluate to zero.

We can them compute a related quantity, called the $\textit{cost}$
$$C[W^{[1]}, b^{[1]}, W^{[2]}, b^{[2]}] = \frac{1}{m}\displaystyle\sum_{i=1}^{m}\mathcal{L}(\hat{y}_{i}, y_{i})$$ which is just the average loss for each event in our training set and gives us a sense of how well our model is performing

In [ ]:
def compute_cost(A2, true_labels, parameters):
    """
    For us, the cost is the average value of the binary crossentropy for each event
    """
    m = true_labels.shape[1]
    
    cost  = (-1/m)* ( np.dot(true_labels, np.log(A2.T)) + np.dot((1-true_labels), np.log(1-A2.T)) )
    cost = np.squeeze(cost) #(n,1) -> (n,)
    
    return cost

# Back Propagation

The goal of training our neural network is to have its predictions get better and better with each iteration. To do this, we need a way of updating the parameters in our network---the weight matrices and biases---depending on how well our network performed. 

Given that we rigidly defined the cost function, we know how it depends on every parameter in our network. So, we can work our ways backwards to figure out how we the cost depends on each parameter in the network. Diagramatically, we can think of this in the following way:

<img src='Images/back_prop.png'>

where $\frac{\partial C}{\partial W^{[2]}}$ tells us how $C$ depends on $W^{[2]}$. To actually compute this, we need to use the chain rule, which gives: 
$$ \frac{\partial C}{\partial W^{[2]}} = \frac{\partial C}{\partial A^{[2]}} \frac{\partial A^{[2]}}{\partial Z^{[2]}} \frac{\partial Z^{[2]}}{\partial W^{[2]}} =: \textrm{"dW2"} $$
We'll then need to repeat a similar calculation for $\textrm{"db2"} = \frac{\partial C}{\partial b^{[2]}}$, $\textrm{"dW1"} = \frac{\partial C}{\partial W^{[1]}}$, and $\textrm{"db1"} = \frac{\partial C}{\partial b^{[1]}}$

For the sake of time, these gradients are provided below, but try and go back and calculate each one to make sure you understand where they came from (you may find it easier to compute these gradients with respect to the loss (single event), rather than the cose (all events)). 

In [ ]:
def back_prop(parameters, cache, input_data, true_labels):
    m = input_data.shape[1]

    W1 = parameters["W1"]
    W2 = parameters["W2"]

    A1 = cache["A1"]
    A2 = cache["A2"]

    dZ2 = A2 - true_labels
    dW2 = (1/m)*np.dot(dZ2, A1.T)
    db2 = (1/m)*np.sum(dZ2, axis=1, keepdims=True)
    dZ1 = (1/m)*np.multiply(np.dot(W2.T, dZ2),(1-np.power(A1, 2)))
    dW1 = (1/m)*np.dot(dZ1, input_data.T)
    db1 = (1/m)*np.sum(dZ1, axis=1, keepdims=True)

    grads = {"dW1":dW1, "db1":db1, "dW2":dW2, "db2":db2}

    return grads

To actually update all of the parameters, we use a $\textit{gradient descent}$ algorithm. The easiest way to explain this is diagramatically (image from https://towardsdatascience.com/quick-guide-to-gradient-descent-and-its-variants-97a7afb33add):

<img src='Images/1*iNPHcCxIvcm7RwkRaMTx1g.jpeg'>

We've computed the gradients of our cost function for each parameter, so we know how each parameter will affect the model's cost function. We then want to take a step of a certain size to a new value of each parameter, with that step being in the direction of the minimum value of the cost function. The size of this step, the $\textit{learning rate}$, is another hyperparameter that we are free to set ourselves, like the number of hidden layers and the number of nodes in each hidden layer.

In [ ]:
def update_parameters(parameters, grads, learning_rate):
    """
    Given the model parameters, gradients, and learning rate, we can update each parameter as 
    parameter = parameter-learning_rate*gradient
    """
    
    W1 = parameters["W1"]
    b1 = parameters["b1"]
    W2 = parameters["W2"]
    b2 = parameters["b2"]
    
    dW1 = grads["dW1"]
    db1 = grads["db1"]
    dW2 = grads["dW2"]
    db2 = grads["db2"]
    
    W1 = W1 - learning_rate*dW1
    b1 = b1 - learning_rate*db1
    W2 = W2 - learning_rate*dW2
    b2 = b2 - learning_rate*db2
    
    parameters = {"W1":W1, "b1":b1, "W2":W2, "b2":b2}
    
    return parameters

# Putting it all together

Now that we know how to define the model architecture, propagate our input data forward to get the model's guess at the correct label, and update all of the parameters in our network based on how closely the model's predicted labels were to the true labels, let's put everything together and actually train our neural network.

We'll train our network for 1000 $\textit{epochs}$, or pass throughs of our entire training set. 

In [ ]:
def nn_model(input_data, true_labels, n_hidden, num_iterations, learning_rate, print_cost=True):
    """
    Start by defining the layer sizes and initializing the parameters. Then carry out the forward propagation, 
    compute the loss and accuracy (and append those to separate lists). Compute the gradients, and finally update
    the model parameters. Every 100 iterations, print the loss and accuracy
    """
    n_inputs = nodes_per_layer(input_data, true_labels)[0]
    n_outputs = nodes_per_layer(input_data, true_labels)[2]
    
    parameters = initialize_parameters(n_inputs, n_hidden, n_outputs)
    cost_list = []
    accuracy_list = []
    for i in range(num_iterations):
        A2, cache = forward_prop(input_data, parameters)
        cost = compute_cost(A2, true_labels, parameters)
        predictions = (A2 > 0.5)
        accuracy = model_accuracy(true_labels[0], predictions[0])
        cost_list.append(cost)
        accuracy_list.append(accuracy)
        grads = back_prop(parameters, cache, input_data, true_labels)
        parameters = update_parameters(parameters, grads, learning_rate)
        if print_cost and i%100==0:
            print(f'After {i} iterations, the model\'s cost is {np.round(cost, 5)} and its accuracy is {np.round(accuracy, 2)}%')
    
    return parameters, cost_list, accuracy_list

In [ ]:
def predict(parameters, input_data):
    A2, cache = forward_prop(input_data, parameters)
    predictions = (A2 > 0.5)
    
    return prediction

In [ ]:
start = perf_counter()
parameters, cost_list, acc_list = nn_model(training_data, training_labels, n_hidden=12, 
                                           num_iterations=1001, learning_rate=10, print_cost=True)
end = perf_counter()

print(f"Our model took {np.round(end-start, 1)} seconds to train")

In [ ]:
# plot training cost, accuracy
fig, ax = plt.subplots(1,2,figsize=(8,4))

ax[0].plot(cost_list)
ax[0].set_xlabel('Epoch')
ax[0].set_ylabel('Cost')

ax[1].plot(acc_list)
ax[1].set_xlabel('Epoch')
ax[1].set_ylabel('Accuracy')

In [ ]:
# how accurate is out model when classifying data it hasn't seen before?


# Simplifying the process with Keras

While the above example is useful for de-mystifying machine learning and gaining an appreciation for what's going on under the hood, it won't be what you use (or want to use) in practice. 

Keras (https://keras.io) is a powerful, easy-to-use machine learning API built on top of TensorFlow2 (https://www.tensorflow.org). Here, we'll use Keras to build a neural network with an identical architecture, train that model on the same data, and see how it compares to the network we wrote from scratch.

In [ ]:
import tensorflow as tf

In [ ]:
# define and train a keras model

input_layer = tf.keras.layers.Input((training_data.T.shape[1],))
hidden_layer = tf.keras.layers.Dense(4, activation='tanh')(input_layer)
output_layer = tf.keras.layers.Dense(1, activation='sigmoid')(hidden_layer)

keras_model = tf.keras.Model(inputs=input_layer, outputs=output_layer)
keras_model.compile(loss='binary_crossentropy', optimizer=tf.keras.optimizers.Adam(1e-3), metrics=['acc'])

history = keras_model.fit(training_data.T, training_labels.T, verbose=1, validation_split=0.15, 
                          epochs=50, batch_size=256)

In [ ]:
# how does the model we instantiated with Keras compare to the one we made from scratch?

fig, ax = plt.subplots(1,2, figsize=(8,4), constrained_layout=True)

ax[0].plot(cost_list, label='NN From Scratch')
ax[0].plot(history.history['loss'], label='Keras NN')
ax[0].set_xlabel('Epoch', fontsize='x-large')
ax[0].set_ylabel('Cost', fontsize='x-large')
ax[0].legend(loc='upper right', fontsize='large')

ax[1].plot(0.01*np.array(accuracy_list), label='NN from Scratch')
ax[1].plot(history.history['acc'], label='Keras NN')
ax[1].set_xlabel('Epoch', fontsize='x-large')
ax[1].set_ylabel('Accuracy', fontsize='x-large')
ax[1].legend(loc='lower right', fontsize='large');